In [ ]:
import requests
import csv
import json
import os
import schedule
import time

url = 'http://api.olhovivo.sptrans.com.br/v0'
s = requests.session()

In [ ]:
def renew_token():
    s.post(url+'/Login/Autenticar?token='+os.getenv('API_KEY'), timeout=3)

In [ ]:
def get_cod_linha( linha ):
    res = s.get(url+'/Linha/Buscar?termosBusca='+linha)
    ret = []
    for linha in res.json():
        ret.append(linha['CodigoLinha'])

    return ret

In [ ]:
# Returns json text ready to be saved as file
def get_posicao( cod_linha ):
    try:
        res = s.get(url+'/Posicao?codigoLinha='+str(cod_linha), timeout=3)
        return res.text
    except Exception:
        return ''

In [ ]:
def get_data():
    renew_token()
    for linha in cod_linhas:
        with open('/data/'+str(linha)+'-'+str(time.time())+'.json', 'w') as f:
            f.write(get_posicao(linha))

In [ ]:
renew_token()
cod_linhas = []
with open('../dados/gtfs-20170411/routes.txt', 'r') as f:
    routes = csv.reader(f)
    next(routes, None)  # ignore header
    for line in routes:
        for cod in get_cod_linha(line[0]):
            cod_linhas.append(cod)

In [ ]:
cod_linhas = set(cod_linhas)  # remove duplicate entries

In [ ]:
schedule.every(1).minutes.do(get_data)

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)